In [3]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import Detectors.Deep_Learning.AutoEncoders.utils
reload(Detectors.Deep_Learning.AutoEncoders.utils)
from Detectors.Deep_Learning.AutoEncoders.utils import SeqDataset,train_epoch,eval_data,train_model

import flow.visualize.visualize_ring as visualize_ring
reload(visualize_ring)
from flow.visualize.visualize_ring import get_measured_leader,get_rel_dist_to_measured_leader,get_vel_of_measured_leader


import torch
print(torch.__version__)

# Anti-Flow specific functions for  detection:

from Detectors.Deep_Learning.AutoEncoders.utils import sliding_window
from Detectors.Deep_Learning.AutoEncoders.cnn_lstm_ae import CNNRecurrentAutoencoder

import utils
reload(utils)
from utils import Bando_OVM_FTL

import os

import Adversaries.controllers.car_following_adversarial
from Adversaries.controllers.car_following_adversarial import FollowerStopper_Overreact
from Adversaries.controllers.car_following_adversarial import ACC_Benign

print('Libraries loaded.')

1.7.1
Libraries loaded.


In [4]:
from flow.networks.ring import RingNetwork
from flow.core.params import VehicleParams

from flow.controllers.car_following_models import IDMController #Human driving model
from flow.controllers.routing_controllers import ContinuousRouter #Router that keeps vehicles on the ring-road

from flow.controllers.lane_change_controllers import AILaneChangeController,StaticLaneChanger


from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import TrafficLightParams
from flow.envs.ring.accel import AccelEnv
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS
from flow.core.params import SumoParams
from flow.core.params import EnvParams
from flow.core.params import SumoCarFollowingParams
from flow.core.experiment import Experiment

print('Flow packages imported.')

Flow packages imported.


In [11]:
def run_ring_sim_variable_cfm(ring_length=300,
    driver_controller_list=None,
    lane_change_controller_list=None,
    num_lanes=1,
    sim_time=500):

    #Simulation parameters:
    time_step = 0.1 #In seconds, how far each step of the simulation goes.
    emission_path = 'data' #Where csv is stored
    want_render = True #If we want SUMO to render the environment and display the simulation.
    sim_horizon = int(np.floor(sim_time/time_step)) #How many simulation steps will be taken -> Runs for 300 seconds

    #initialize the simulation using above parameters:
    traffic_lights = TrafficLightParams() #This is empty, so no traffic lights are used.
    initial_config = InitialConfig(shuffle=True,spacing="uniform", perturbation=1) #Vehicles start out evenly spaced.
    vehicles = VehicleParams() #The vehicles object will store different classes of drivers:
    sim_params = SumoParams(sim_step=time_step, render=want_render, emission_path=emission_path) #Sets the simulation time-step and where data will be recorded.
    env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)
    net_params = NetParams(additional_params={'length':ring_length,
                                              'lanes':num_lanes,
                                              'speed_limit': 30,
                                              'resolution': 40})

    # Insert different vehicles:

    if(driver_controller_list is None and lane_change_controller_list is None):
        print('Running IDM.')
        #Default to the IDM if otherwise controllers not specified:
        vehicles.add("idm_driver",
                     acceleration_controller=(IDMController, {'noise':0.1}),
                     routing_controller=(ContinuousRouter, {}),
                     lane_change_controller=(StaticLaneChanger, {}),
                     num_vehicles=num_human_drivers)

    elif(driver_controller_list is not None and lane_change_controller_list is None):
        print('Number unique drivers: '+str(len(driver_controller_list)))
        for driver in driver_controller_list:
            label = driver[0]
            cfm_controller = driver[1]
            num_vehicles = driver[2]

            vehicles.add(label,
                acceleration_controller = cfm_controller,
                routing_controller=(ContinuousRouter, {}),
                lane_change_controller=(StaticLaneChanger, {}),
                num_vehicles=num_vehicles)
            
    elif(driver_controller_list is not None and lane_change_controller_list is not None):
        print('Number unique drivers: '+str(len(driver_controller_list)))
        print('Unique lane-change dynamics present.')
        num_drivers = len(driver_controller_list)
        for i in range(num_drivers):
            driver = driver_controller_list[i]
            
            label = driver[0]
            cfm_controller = driver[1]
            num_vehicles = driver[2]
            
            lc_controller = lane_change_controller_list[i]

            vehicles.add(label,
                acceleration_controller = cfm_controller,
                routing_controller=(ContinuousRouter, {}),
                lane_change_controller=lc_controller,
                num_vehicles=num_vehicles)


    #initialize the simulation:
    flow_params = dict(
        exp_tag='ring_variable_cfm',
        env_name=AccelEnv,
        network=RingNetwork,
        simulator='traci',
        sim=sim_params,
        env=env_params,
        net=net_params,
        veh=vehicles,
        initial=initial_config,
        tls=traffic_lights,
    )

    flow_params['env'].horizon = sim_horizon
    exp = Experiment(flow_params)
    print('Running ring simulation, ring length: '+str(ring_length))
    
    sim_res_list = exp.run(1, convert_to_csv=True)
    
    return sim_res_list

print('Defined multi-lane ring sim.')

Defined multi-lane ring sim.


In [9]:
driver_controller_list = []

sim_time = 500
ring_length = 400
num_lanes = 2


a_mean=0.8
b_mean=20.0
s0_mean=1.0
s1_mean=2.0
Vm_mean=15.0

for i in range(30):
    a = a_mean + np.random.normal(0,0.1)
    b = b_mean + np.random.normal(0,0.5)
    s0 = s0_mean + np.random.normal(0,0.2)
    s1 = s1_mean + np.random.normal(0,0.2)
    Vm = Vm_mean + np.random.normal(0,1.5)

    label = 'bando_ftl_ovm_a'+str(np.round(a,2))+'_b'+str(np.round(b,2))+'_s0'+str(np.round(s0,2))+'_s1'+str(np.round(s1,2))+'_Vm'+str(np.round(Vm,2))
    cfm_controller = (Bando_OVM_FTL,{'a':a,'b':b,'s0':s0,'s1':s1,'Vm':Vm,'noise':0.1})
    driver_controller_list.append([label,cfm_controller,1])


k_1_mean = 1.0
k_2_mean = 1.0
h_mean = 1.5
V_m_mean = 15.0

for i in range(10):
    k_1 = k_1_mean + np.random.normal(0,0.2)
    k_2 = k_2_mean + np.random.normal(0,0.2)
    h = h_mean + np.random.normal(0,0.2)
    V_m = V_m_mean + np.random.normal(0,1.0)
    
    label = 'ACC_k_1'+str(np.round(k_1,2))+'_k_2'+str(np.round(k_2,2))+'_h'+str(np.round(h,2))+'_V_m'+str(np.round(V_m,2))
    cfm_controller = (ACC_Benign,{'k_1':k_1,'k_2':k_2,'h':h,'V_m':V_m})
    driver_controller_list.append([label,cfm_controller,1])

    
lane_change_controller_list = []
for i in range(40):
    lane_change_controller_list.append((StaticLaneChanger, {}))
    

print('Finished initializing sim dynamics.')    

Finished initializing sim dynamics.


In [12]:
sim_res_list_no_lane_change = run_ring_sim_variable_cfm(ring_length=ring_length,
    driver_controller_list=driver_controller_list,
    lane_change_controller_list=lane_change_controller_list,
    num_lanes=num_lanes,
    sim_time=sim_time)

Number unique drivers: 40
Unique lane-change dynamics present.
Running ring simulation, ring length: 400
Round 0, return: 4295.24719143128
data/ring_variable_cfm_20220313-1209451647191385.250991-0_emission.csv data
Average, std returns: 4295.24719143128, 0.0
Average, std velocities: 9.924239449116559, 0.0
Average, std outflows: 0.0, 0.0
Total time: 113.14419198036194
steps/second: 69.74613057369939


In [13]:
lane_change_controller_list = []
for i in range(40):
    left_delta=0.5 + np.random.normal(0,0.1)
    right_delta=0.5+ np.random.normal(0,0.1)
    left_beta=np.min([0.5,1.5+ np.random.normal(0,0.1)])
    right_beta=np.min([0.5,1.5+ np.random.normal(0,0.1)])
    switching_threshold = 5.0
    lane_change_controller_list.append((AILaneChangeController, {'left_delta':left_delta,
                                                                'right_delta':right_delta,
                                                                'left_beta':left_beta,
                                                                'right_beta':right_beta}))
    
print('Redifed lane-change dyamics.')

Redifed lane-change dyamics.


In [14]:
sim_res_list_with_lane_change = run_ring_sim_variable_cfm(ring_length=ring_length,
    driver_controller_list=driver_controller_list,
    lane_change_controller_list=lane_change_controller_list,
    num_lanes=num_lanes,
    sim_time=sim_time)

Number unique drivers: 40
Unique lane-change dynamics present.
Running ring simulation, ring length: 400


KeyError: 'lane_leaders'